In [32]:
import glob
import os
import random
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm.notebook import trange, tqdm

In [ ]:
# for tile_heights folder for each city, calculate the rmse for summer, spring, winter 

files = glob.glob('data/tile_heights/*.csv')

for file in tqdm(files, total=len(files), leave=False):
    file = file.replace('\\','/')
    tiles = pd.read_csv(file)
    city = file.split('/')[-1].split('.')[0].split('-')[0]

    # create three columns for summer, spring, winter and add rmse values for each tile for three seasons
    tiles['summer'], tiles['spring'], tiles['winter'] = 0, 0, 0

    # iter tiles df
    for index, row in tqdm(tiles.iterrows(), total=tiles.shape[0], leave=False):
        # get i, j
        i = int(row['i'])
        j = int(row['j'])
        seasons = ['summer', 'spring', 'winter']
        for season in seasons:
            
            heights = np.empty([256, 256])
            img = cv2.imread('data/heights_new/%s/16/%d/%d.png'%(city,i,j)) 
            heights = img[:,:,0]

            img = cv2.imread('data/shadows_new/%s-%s/16/%d/%d.png'%(city,season,i,j)) 
            values_gan = img[:,:,0]
            values_gan[heights>0] = 0

            img = cv2.imread('data/shadows/%s-%s/16/%d/%d.png'%(city,season,i,j))
            values_true = img[:,:,0]
            values_true[heights>0] = 0

            target = values_true/255    #[:,128:-128,128:-128,:]
            target = target.astype(np.float32)
            prediction = values_gan/255     #[:,128:-128,128:-128,:]
            prediction = prediction.astype(np.float32)

            mse = np.mean((target-prediction) ** 2)
            rmse = np.sqrt(mse)
            tiles.at[index, season] = rmse # in place update of row[season]

    tiles.to_csv('data/tile_heights/%s-16.csv'%city, index=False)

In [3]:
# combine all cities together in one dataframe
files = glob.glob('data/tile_heights/*.csv')
df = pd.DataFrame()
for file in tqdm(files, total=len(files), leave=False):
    file = file.replace('\\','/')
    city = file.split('/')[-1].split('.')[0].split('-')[0]
    tiles = pd.read_csv(file)
    tiles['city'] = city
    # without append
    df = pd.concat([df, tiles], ignore_index=True)

  0%|          | 0/14 [00:00<?, ?it/s]

In [4]:
df['avg'] = df[['summer', 'spring', 'winter']].mean(axis=1)
df

,i,j,height,summer,spring,winter,city,avg
0,14951,26964,11.102304,0.024548,0.022893,0.030387,aus,0.025943
1,14951,26965,8.771652,0.027654,0.025744,0.032588,aus,0.028662
2,14951,26966,7.091804,0.036468,0.035041,0.041422,aus,0.037644
3,14951,26967,5.792377,0.054950,0.055047,0.067578,aus,0.059192
4,14951,26968,6.835874,0.039291,0.039068,0.047510,aus,0.041956
...,...,...,...,...,...,...,...,...
22737,58239,25814,6.470588,0.140046,0.156544,0.186568,tok,0.161053
22738,58239,25815,6.470588,0.151712,0.189622,0.290870,tok,0.210735
22739,58239,25816,6.470588,0.115214,0.142628,0.180287,tok,0.146043
22740,58239,25817,6.470588,0.047356,0.062975,0.046908,tok,0.052413


In [5]:
# find quantiles of avg rmse
rmse_quantiles = df['avg'].quantile([0, 0.25, 0.5, 0.75, 1]).to_list()
rmse_iqr = rmse_quantiles[3] - rmse_quantiles[1]
print(rmse_quantiles)
print(rmse_iqr* 1.5)

[0.0013154229576077666, 0.030090603977441756, 0.060380268841981846, 0.08397920181353884, 0.4377887348333995]
0.08083289675414562


In [6]:
height_quantiles = df['height'].quantile([0, 0.25, 0.5, 0.75, 1]).to_list()
height_iqr = height_quantiles[3] - height_quantiles[1]
print(height_quantiles)

[0.0, 6.470588235294118, 6.470588235294119, 7.7298641889929645, 228.59184781473823]


In [8]:
# df[(df['avg']<=rmse_quantiles[3]) & (df['height']<=height_quantiles[2]) & (df['city'] == 'chi')]

In [9]:
colors = [(1, 1, 1), (0, 0, 0)]
cmap_g = LinearSegmentedColormap.from_list('colorscale', colors, N=256)

colors = [(1, 1, 1), (254/255,204/255,92/255), (253/255,141/255,60/255), (240/255,59/255,32/255), (189/255,0,38/255)]
cmap_r = LinearSegmentedColormap.from_list('colorscale', colors, N=100)

In [ ]:
test = df[(df['avg']<=rmse_quantiles[3]) & (df['city'] == 'tok')]
# print(len(test))
# (df['avg']<=rmse_quantiles[3])
# (df['avg']>rmse_iqr* 1.5)
# & (df['height']<=height_quantiles[2])

# iter test. pick random 50
# test = test.sample(n=50, replace=False)
for index, row in tqdm(test.iterrows(), total=test.shape[0], leave=False):
    # get i, j
    i = int(row['i'])
    j = int(row['j'])
    height = float(row['height'])
    avg = float(row['avg'])

    print(i,j, height, avg)
    city = 'tok'
    season = 'winter'

    heights = np.empty([256, 256])
    img = cv2.imread('data/heights_new/%s/16/%d/%d.png'%(city,i,j))
    heights = img[:,:,0]


    img = cv2.imread('data/shadows_new/%s-%s/16/%d/%d.png'%(city,season,i,j))
    values_gan = img[:,:,0]
    values_gan[heights>0] = 0

    img = cv2.imread('data/shadows/%s-%s/16/%d/%d.png'%(city,season,i,j))
    values_true = img[:,:,0]
    values_true[heights>0] = 0

    # find contours
    contours, hierarchy = cv2.findContours(heights, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)>100):

        # subplot the three
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
        ax1.imshow(heights/np.max(heights), cmap=cmap_g)
        ax2.imshow(values_gan, cmap=cmap_r)
        ax3.imshow(values_true, cmap=cmap_r)


        plt.show()


In [29]:
# chi:
# small error: 16798 24361, 16798 24351, 16825 24384
# large error: 16814 24372, 16818 24354, 16815 24352
# small error new: 16796 24357, 16797 24365, 16803 24348, 16803 24348, 16806 24333, 16806 24337

# arr = [['chi', 16798, 24361], ['chi', 16798, 24351], ['chi', 16825, 24384], ['chi', 16814, 24372], ['chi', 16818, 24354], ['chi', 16815, 24352]]

# arr only with small error new:
arr = [['chi', 16796, 24357], ['chi', 16797, 24365], ['chi', 16803, 24348], ['chi', 16803, 24348], ['chi', 16806, 24333], ['chi', 16806, 24337]]

In [30]:
# tok:
# small error 58195 25809, 58233 25819, 58218 25808
# large error 58206 25810, 58212 25805, 58198 25805, 58207 25807, 58204 25820

# small error new: 58221 25806, 58218 25808, 58204 25799

# extend arr with tok cases
# arr.extend([['tok', 58195, 25809], ['tok', 58233, 25819], ['tok', 58218, 25808], ['tok', 58206, 25810], ['tok', 58212, 25805], ['tok', 58198, 25805], ['tok', 58207, 25807], ['tok', 58204, 25820]])

# arr only with small error new:
arr.extend([['tok', 58221, 25806], ['tok', 58218, 25808], ['tok', 58204, 25799]])

In [31]:
# bue:
# small error: 22140 39495, 22138 39491, 22144 39496
# large error: 22141 39488, 22128 39503, 22143 39491, 22141 39489, 

# small error new: 22143 39495, 22143 39496, 22138 39493

# arr.extend([['bue', 22140, 39495], ['bue', 22138, 39491], ['bue', 22144, 39496], ['bue', 22141, 39488], ['bue', 22128, 39503], ['bue', 22143, 39491]])

# arr only with small error new:
arr.extend([['bue', 22143, 39495], ['bue', 22143, 39496], ['bue', 22138, 39493]])

In [33]:
# Folders like chi_16798_24351/summer, chi_16798_24351/spring, chi_16798_24351/winter
# And then inside each folder, put heights, values_gan, values_true

seasons = ['summer', 'spring', 'winter']
for city, i, j in tqdm(arr, total=len(arr), leave=False):
    for season in seasons:
        heights = np.empty([256, 256])
        img = cv2.imread('data/heights_new/%s/16/%d/%d.png'%(city,i,j))
        heights = img[:,:,0]


        img = cv2.imread('data/shadows_new/%s-%s/16/%d/%d.png'%(city,season,i,j))
        values_gan = img[:,:,0]
        values_gan[heights>0] = 0

        img = cv2.imread('data/shadows/%s-%s/16/%d/%d.png'%(city,season,i,j))
        values_true = img[:,:,0]
        values_true[heights>0] = 0

        # save to folder. plt axis off
        folder = 'data/plots/fig4/%s/general/%s_%d_%d/%s'%(city, city, i,j,season)
        if not os.path.exists(folder):
            os.makedirs(folder)

        plt.imshow(heights, cmap=cmap_g)
        plt.axis('off')
        plt.savefig('%s/heights.png'%folder, bbox_inches='tight', pad_inches=0)
        plt.close()

        plt.imshow(values_gan, cmap=cmap_r)
        plt.axis('off')
        plt.savefig('%s/values_gan.png'%folder, bbox_inches='tight', pad_inches=0)
        plt.close()

        plt.imshow(values_true, cmap=cmap_r)
        plt.axis('off')
        plt.savefig('%s/values_true.png'%folder, bbox_inches='tight', pad_inches=0)
        plt.close()


  0%|          | 0/12 [00:00<?, ?it/s]